In [15]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
from tqdm import tqdm

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from surface_vae_trainer import SurfaceVAETrainer
from dupire_pinn_trainer import DupirePINNTrainer
from evolutionary_algorithm import EvolutionaryAlgorithm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [17]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

prior_covariance_matrix = smoothness_prior.prior_covariance()
prior_eigenvalues = np.sort(np.linalg.eigvalsh(prior_covariance_matrix))[::-1].copy()

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(1000)

# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

hidden_dim = 64
n_layers = 8
batch_size = 100
pde_loss_coefficient = 650.0
maturity_zero_loss_coefficient = 800.0
strike_zero_loss_coefficient = 40.0
strike_infinity_loss_coefficient = 200.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 200
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
torch.manual_seed(0)
pinn_trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

pinn_trainer.load_model()

pinn_trainer.dupire_price_prediction_loss(
    torch.tensor(sampled_surface_coefficients, device=device, dtype=torch.float32),
    call_option_dataset["Call Option Price"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
)
print()    

PINN Model loaded from models/pinn_model.pth.



In [18]:
latent_dim = 40  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

torch.manual_seed(2)
vae_trainer = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

vae_trainer.load_model()

VAE Model loaded from models/vae_model.pth.


In [31]:
# Define the parameter grid
# population_sizes = [10, 100, 1000]
population_sizes = [100]
mutation_strengths = [0.1, 1.0, 10.0]
selection_pressure_parameters = [0.01, 0.1, 0.3, 0.5, 0.7]
n_generations = 1000
truncation_clip = 3

# Store results for each configuration
results = []

for population_size, mutation_strength, selection_pressure_parameter in tqdm(
    product(population_sizes, mutation_strengths, selection_pressure_parameters),
    total=len(population_sizes) * len(mutation_strengths) * len(selection_pressure_parameters),
    desc="Processing combinations"
):
    # Initialize Evolutionary Algorithm with the current parameters
    torch.manual_seed(2)
    evolution_algo = EvolutionaryAlgorithm(
        vae_trainer=vae_trainer,
        pinn_trainer=pinn_trainer,
        latent_diagonal=latent_diagonal,
        population_size=population_size,
        mutation_strength=mutation_strength,
        selection_pressure_parameter=selection_pressure_parameter,
        n_generations=n_generations,
        truncation_clip=truncation_clip
    )

    # Run the optimization
    evolution_algo.optimize()

    # Metrics calculation
    final_best_fitness = evolution_algo.optimization_history["Best Fitness"][-1]  # Best fitness
    initial_fitness = evolution_algo.optimization_history["Best Fitness"][0]
    n_generation = n_generations

    # Calculate the diversity
    population_variance = np.var(evolution_algo.population.cpu().numpy(), axis=0)
    diversity = np.sum(population_variance / latent_diagonal)

    # Store the results for this configuration
    results.append({
        "population_size": population_size,
        "mutation_strength": mutation_strength,
        "selection_pressure_parameter": selection_pressure_parameter,
        "final_best_fitness": final_best_fitness,
        "diversity": diversity,
    })

Processing combinations: 100%|██████████| 15/15 [02:10<00:00,  8.70s/it]


In [32]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Rank the configurations based on each metric
results_df["rank_best_fitness"] = results_df["final_best_fitness"].rank(ascending=False)
results_df["rank_diversity"] = results_df["diversity"].rank(ascending=False)

# Calculate the average rank
results_df["average_rank"] = results_df[["rank_best_fitness", "rank_diversity"]].mean(axis=1)

# Sort by the average rank (lower is better)
results_df_sorted = results_df.sort_values(by="average_rank", ascending=True)

# Display the sorted results
results_df_sorted

,population_size,mutation_strength,selection_pressure_parameter,final_best_fitness,diversity,rank_best_fitness,rank_diversity,average_rank
12,100,10.0,0.30,-0.577135,260.217485,6.0,3.0,4.50
14,100,10.0,0.70,-0.567234,173.129296,5.0,5.0,5.00
10,100,10.0,0.01,-0.592927,263.160720,9.5,1.0,5.25
13,100,10.0,0.50,-0.579160,253.180994,7.0,4.0,5.50
11,100,10.0,0.10,-0.592927,260.787584,9.5,2.0,5.75
8,100,1.0,0.50,-0.555634,77.703173,4.0,9.0,6.50
9,100,1.0,0.70,-0.524825,48.122904,3.0,10.0,6.50
3,100,0.1,0.50,-0.524333,2.219683,2.0,14.0,8.00
4,100,0.1,0.70,-0.498038,1.550894,1.0,15.0,8.00
7,100,1.0,0.30,-0.581921,94.704854,8.0,8.0,8.00


In [41]:
# Define evolutionary algorithm hyperparameters
population_size = 100 # Population size
mutation_strength = 1.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 1000  # Number of generations for the evolutionary process
truncation_clip = 3
torch.manual_seed(2)
# Initialize the EvolutionaryAlgorithm
evolution_algo = EvolutionaryAlgorithm(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    truncation_clip=truncation_clip
)

# Run the optimization process
evolution_algo.optimize()

# Create the figure
fig = go.Figure()

# Calculate the diversity
population_variance = np.var(evolution_algo.population.cpu().numpy(), axis=0)
diversity = np.sum(population_variance / latent_diagonal)

optimization_history = pd.DataFrame(evolution_algo.optimization_history)
best_fitness = optimization_history["Best Fitness"].iloc[-1]

# Add trace for best fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["Best Fitness"]) + 1)),
    y=optimization_history["Best Fitness"],
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue', width=3),
    marker=dict(size=6, symbol='circle'),
    hovertemplate='<b>Iteration %{x}</b><br>Best Fitness: %{y:.4f}<extra></extra>'
))

# Add trace for average fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["Average Fitness"]) + 1)),
    y=optimization_history["Average Fitness"],
    mode='lines+markers',
    name='Average Fitness',
    line=dict(color='firebrick', width=3, dash='dash'),
    marker=dict(size=6, symbol='square'),
    hovertemplate='<b>Iteration %{x}</b><br>Average Fitness: %{y:.4f}<extra></extra>'
))

# Add trace for avg fitness - 1.96 * std fitness
fig.add_trace(go.Scatter(
    x=list(range(1, len(optimization_history["0.05 Percentile Fitness"]) + 1)),
    y=optimization_history["0.05 Percentile Fitness"],
    mode='lines+markers',
    name='0.05 Percentile Fitness',
    line=dict(color='green', width=3, dash='dot'),
    marker=dict(size=6, symbol='triangle-up'),
    hovertemplate='<b>Iteration %{x}</b><br>0.05 Percentile Fitness: %{y:.4f}<extra></extra>'
))

# Add an annotation for best fitness and diversity
fig.add_annotation(
    xref='paper', yref='paper', 
    x=0.96, y=0.05,
    text=f"Best Fitness: {best_fitness:.4f}<br>Diversity: {diversity:.4f}",
    showarrow=False,
    # font=dict(
    #     size=14,
    #     color="black"
    # ),
    align="left",
    # bordercolor="black",
    borderwidth=2,
    borderpad=4,
    bgcolor="white",
    opacity=1.
)

# Update layout to make the plot detailed and visually appealing
fig.update_layout(
    title="Initial Evolutionary Optimization History",
    xaxis_title="Generation",
    yaxis_title="Fitness Value",
    legend=dict(
        x=0.75,
        y=0.15,
    ),
    hovermode='x unified',
    width=900,
    height=900,
)

# Show the plot
fig.show()
fig.write_image('figs/initial_evolutionary_optimization_history.png', format='png', scale=4, width=900, height=900)

100%|██████████| 1000/1000 [00:05<00:00, 167.30it/s]


In [43]:
# Sample 4 surface coefficients using the smoothness prior
n_samples = 4
fine_tune_surface_coefficients = evolution_algo.final_population.values
sampled_coefficients = fine_tune_surface_coefficients[:4]

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

# Create the 3x3 subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f"Sample Surface {i+1}" for i in range(n_samples)],
    specs=[[{'type': 'surface'}] * 2] * 2,
    vertical_spacing=0.075, horizontal_spacing=0.001
)

# Loop through the sampled coefficients to generate and plot each surface
for idx, coefficients in enumerate(sampled_coefficients):
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = np.array([
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in hypothetical_product_grid
    ]).reshape(len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list))

    # Add the surface plot to the subplot
    fig.add_trace(
        go.Surface(
            x=hypothetical_strikes_grid,
            y=hypothetical_maturities_grid,
            z=surface_volatilities,
            colorscale='Cividis',
            showscale=False  # Hide the scale on individual plots
        ),
        row=row, col=col
    )

for i in range(1, 3):  # Rows
    for j in range(1, 3):  # Columns
        fig.update_scenes(
            dict(
                xaxis_title="Strike Price",
                yaxis_title="Time to Maturity",
                zaxis_title="Implied Volatility",
                camera=dict(
                    eye=dict(x=1.5, y=1.5, z=1.5)  # Controls the zoom level, adjust these to zoom out
                ),
                aspectratio=dict(x=1, y=1, z=0.8)
            ),
            row=i, col=j
        )    

# Update the layout of the figure
fig.update_layout(
    title="Initial Latent Evolutionary Optimization Top Surfaces",
    height=900, width=900, title_x=0.5,
)

# Show the plot
fig.show()
fig.write_image('figs/initial_evolutionary_optimization_top_surfaces.png', format='png', scale=4, width=900, height=900)